<a href="https://colab.research.google.com/github/EunSik312/lecture-deeplearning/blob/main/7%EC%9B%9418%EC%9D%BCyolo12_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yt-dlp ultralytics opencv-python

from google.colab import files
from ultralytics import YOLO
import yt_dlp
import os
import cv2
from IPython.display import Video, display
model = YOLO("yolo12n.pt")
model.info()
results = model.train(data="coco8.yaml", epochs=10, imgsz=640)

# 유튜브 영상 다운로드 함수
def download_youtube_video(url):
    """유튜브 영상을 다운로드하는 함수"""
    output_path = "./downloads"
    os.makedirs(output_path, exist_ok=True)

    ydl_opts = {
        'format': 'best[height<=720]',  # 720p 이하 최고 화질
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            title = info.get('title', 'video')

            # 영상 다운로드
            ydl.download([url])

            # 다운로드된 파일 경로 찾기
            for file in os.listdir(output_path):
                if file.endswith(('.mp4', '.mkv', '.webm')):
                    video_path = os.path.join(output_path, file)
                    print(f"다운로드 완료: {video_path}")
                    return video_path

    except Exception as e:
        print(f"다운로드 오류: {e}")
        return None

# 영상 처리 함수
def process_video(video_path, output_path="./output_video.mp4"):
    """영상에서 객체 탐지를 수행하는 함수"""
    cap = cv2.VideoCapture(video_path)

    # 영상 속성 가져오기
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"영상 정보: {width}x{height}, FPS: {fps}, 총 프레임: {total_frames}")

    # 비디오 작성기 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0

    # 프레임별 처리
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO 객체 탐지 (기존 코드와 동일한 방식)
        results = model(frame)

        # 탐지 결과를 프레임에 그리기
        annotated_frame = results[0].plot()

        # 프레임 저장
        out.write(annotated_frame)

        # 진행률 표시
        if frame_count % 30 == 0:
            progress = (frame_count / total_frames) * 100
            print(f"처리 진행률: {progress:.1f}%")

        frame_count += 1

    # 자원 해제
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    return output_path

# 방법 1: 유튜브 URL 직접 입력
youtube_url = input("유튜브 영상 URL을 입력하세요: ")

if youtube_url:
    # 유튜브 영상 다운로드
    video_path = download_youtube_video(youtube_url)

    if video_path:
        # 다운로드한 영상에 대해 YOLOv11n 모델로 추론 실행
        output_video = process_video(video_path)
    else:
        print("영상 다운로드에 실패했습니다.")

print("\n처리 완료!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

100%|██████████| 5.34M/5.34M [00:00<00:00, 107MB/s]


YOLOv12n summary: 272 layers, 2,603,056 parameters, 0 gradients, 6.7 GFLOPs
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=1

100%|██████████| 433k/433k [00:00<00:00, 16.2MB/s]
Unzipping /content/datasets/coco8.zip to /content/datasets/coco8...: 100%|██████████| 25/25 [00:00<00:00, 3314.61file/s]

Dataset download success ✅ (0.4s), saved to /content/datasets



...

                   all          4         17      0.575      0.361      0.326      0.248



10 epochs completed in 0.019 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.6MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.6MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv12n summary (fused): 159 layers, 2,590,824 parameters, 0 gradients, 6.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


                   all          4         17      0.574       0.36      0.326      0.248
                person          3         10      0.435      0.161       0.26      0.192
                   dog          1          1          1          0      0.142     0.0853
                 horse          1          2      0.472          1      0.497      0.253
              elephant          1          2          0          0          0          0
              umbrella          1          1      0.534          1      0.995      0.895
          potted plant          1          1          1          0     0.0622     0.0622
Speed: 2.4ms preprocess, 392.7ms inference, 0.0ms loss, 6.4ms postprocess per image
Results saved to runs/detect/train
유튜브 영상 URL을 입력하세요: https://www.youtube.com/watch?v=KtBa71KDNkQ
[youtube] Extracting URL: https://www.youtube.com/watch?v=KtBa71KDNkQ
[youtube] KtBa71KDNkQ: Downloading webpage
[youtube] KtBa71KDNkQ: Downloading tv client config
[youtube] KtBa71KDNkQ: Downloa

error: OpenCV(4.12.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
